In [143]:
import cv2
import numpy as np
import os
from random import shuffle
from tqdm import tqdm
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
import csv

In [144]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters 
input_size = 6
hidden_size = 50
num_classes = 15
num_epochs = 10
batch_size = 100
learning_rate = 0.001

TRAIN_DIR = 'D:/PycharmProjects/marchMadness/train'

In [145]:
def read_train_data(file):
    with open(TRAIN_DIR + '/' + file) as fd:
        reader=csv.reader(fd)
        data=[row for idx, row in enumerate(reader) if idx in (0,1,2)]
        for i in data:
            for x in range(0,len(i)):
                i[x] = float(i[x])         
    return data

def labelTM(data):
    label = data[2][0]
    return label

def labelOPP(data):
    label = data[2][1]
    return label

def label_file(file):
    word_label = file.split('.')[-6]
    if word_label == 'win': 
        x = 1
    elif word_label == 'loss': 
        x = 0
    return x

In [147]:
def create_train_data():
    training_data = []
    training_labels = []
    training_labels_tm = []
    training_labels_opp = []
    for file in tqdm(os.listdir(TRAIN_DIR)):
        outcome = label_file(file)
        data = read_train_data(file)
        scoreTM = int(labelTM(data))
        scoreTM = round(scoreTM,-1)
        scoreOPP = int(labelOPP(data))
        scoreOPP = round(scoreOPP,-1)
        training_data.append(torch.FloatTensor(data[:2]).unsqueeze(0))
        training_labels_tm.append(scoreTM)
        training_labels_opp.append(scoreOPP)
        training_labels.append(outcome)
    return training_data, training_labels_tm, training_labels_opp, training_labels

training_data, training_labels_tm, training_labels_opp, training_labels = create_train_data()

100%|████████████████████████████████████████████████████████████████████████| 141404/141404 [00:24<00:00, 5843.03it/s]


In [148]:
training_data = training_data[:-10000]
training_labels = training_labels[:-10000]
training_labels_tm = torch.LongTensor(training_labels_tm[:-10000])
training_labels_opp = torch.LongTensor(training_labels_opp[:-10000])
testing_data = training_data[-10000:]
testing_labels_tm = torch.LongTensor(training_labels_tm[-10000:])
testing_labels_opp = torch.LongTensor(training_labels_opp[-10000:])
testing_labels = torch.LongTensor(training_labels[-10000:])

print('No. of training data: ' + str(len(training_data)))
print('No. of training labels tm: ' + str(len(training_labels_tm)))
print('No. of training labels opp: ' + str(len(training_labels_opp)))
print('No. of training labels outcome: ' + str(len(training_labels)))
print('No. of testing data: ' + str(len(testing_data)))
print('No. of testing labels tm: ' + str(len(testing_labels_tm)))
print('No. of testing labels opp: ' + str(len(testing_labels_opp)))
print('No. of testing labels outcome: ' + str(len(testing_labels)))


No. of training data: 131404
No. of training labels tm: 131404
No. of training labels opp: 131404
No. of training labels outcome: 131404
No. of testing data: 10000
No. of testing labels tm: 10000
No. of testing labels opp: 10000
No. of testing labels outcome: 10000


In [140]:
class MyDataset(Dataset):
    def __init__(self, datas, labels1, labels2, labels3):
        self.datas = datas
        self.labels1 = labels1
        self.labels2 = labels2
        self.labels3 = labels3

    def __getitem__(self, index):
        data, target1, target2, target3 = self.datas[index], self.labels1[index], self.labels2[index], self.labels3[index]
        return data, target1, target2

    def __len__(self):
        return len(self.datas)


train_dataloader = DataLoader(MyDataset(training_data, training_labels_tm, training_labels_opp, training_labels), batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(MyDataset(testing_data, testing_labels_tm, testing_labels_opp, testing_labels), batch_size=batch_size, shuffle=False)


In [141]:
# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        x1 = self.fc1(x)
        x2 = self.fc1(x)
        x3 = self.fc1(x)
        x1 = self.relu(x1)
        x2 = self.relu(x2)
        x3 = self.relu(x3)
        x1 = self.fc2(x1)
        x2 = self.fc2(x2)
        x3 = self.fc2(x3)
        return x1,x2,x3

model = NeuralNet(input_size, hidden_size, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 

In [142]:
# Train the model
total_step = len(train_dataloader)
print(total_step)
for epoch in range(num_epochs):
    for i, (images, target1, target2, target3) in enumerate(train_dataloader):  
        # Move tensors to the configured device
        images = images.reshape(-1, 2*3).to(device)
        target1 = target1.to(device)
        target2 = target2.to(device)
        target3 = target3.to(device)
        
        # Forward pass
        out1, out2, out3 = model(images)
        #print(out1)
        #print('--------------------')
        loss1 = criterion(out1, target1)
        loss2 = criterion(out2, target2)
        loss3 = criterion(out3, target3)
        loss = loss1 + loss2 + loss3
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
.format(epoch+1, num_epochs, i+1, total_step, loss.item()), end='\r')

1315


In [120]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, target1, target2 in test_dataloader:
        images = images.reshape(-1, 2*3).to(device)
        target1 = target1.to(device)
        target2 = target2.to(device)
        out1, out2 = model(images)
        _, predicted1 = torch.max(out1.data, 1)
        _, predicted2 = torch.max(out2.data, 1)
        print('-----------------------------------------------------------------------------------------------------')
        print('score tm: ' + str(target1))
        print('score opp: ' + str(target2))
        print('predicted tm: ' + str(predicted1))
        print('predicted op: ' + str(predicted2))
        print('-----------------------------------------------------------------------------------------------------')
        total += target1.size(0) + target2.size(0)
        correct += (predicted1 == target1).sum().item() + (predicted2 == target2).sum().item()

print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

-----------------------------------------------------------------------------------------------------
score tm: tensor([  85,   64,   91,   73,   59,   98,   76,   68,   52,   85,
          68,   69,   57,   69,   71,   75,   71,   67,   73,   70,
          67,   70,   69,   62,   82,   55,   82,   75,   79,   69,
          59,   69,   81,   78,   76,   63,   66,   53,   75,   76,
          70,   78,   82,   82,   96,   74,  110,   82,   57,   77,
          83,   76,   75,   95,   95,   96,   88,   89,   87,   75,
          71,   94,   80,   74,   83,   92,   81,   69,   98,   83,
          87,   80,   69,   66,   53,   61,   74,   74,   57,   80,
          81,  113,   79,   86,   80,   85,   94,   82,   99,  105,
         103,  104,   81,   91,   82,   88,   80,   88,   72,   63], device='cuda:0')
score opp: tensor([ 79,  50,  63,  37,  44,  64,  74,  66,  49,  84,  67,  54,
         55,  57,  60,  71,  67,  64,  72,  47,  58,  69,  60,  59,
         70,  33,  69,  62,  69,  56,  47, 

-----------------------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------------------
score tm: tensor([  67,   79,   80,   61,   72,   96,   81,   75,   58,   63,
          86,   82,   79,   75,   82,  104,   73,   84,   77,   86,
          82,   82,   85,   79,   72,   81,   71,   82,   68,   74,
          89,   80,   96,   69,   80,   76,   66,   65,   76,   75,
          88,   72,   79,   79,   71,   98,   77,   83,   90,   76,
          75,   90,   64,   75,   73,   85,   69,   67,   72,   76,
          77,   91,   77,   78,   88,   84,  104,   74,   81,   86,
          70,   76,   72,   89,   77,   68,   80,   81,   95,   56,
          88,   87,   77,   57,   82,   68,   97,   74,   85,   84,
          65,   87,   73,   86,   89,   92,   70,   66,   73,   78], device='cuda:0')
score opp: tensor([  64,   72,   51,   58,   65,   95,   80,   69,   51,   61,
         

         70,  56,  74,  70], device='cuda:0')
-----------------------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------------------
score tm: tensor([  69,   61,   92,   86,   99,   85,   79,  102,   85,   88,
          71,   84,   86,   62,   92,   69,   92,   90,   82,   71,
          92,   88,   85,   68,   50,   82,   81,   84,   66,   65,
          79,   94,   63,   75,   70,   68,   81,   88,   52,   78,
          85,   86,   74,   83,   79,   99,   90,   73,   85,   70,
          64,   71,   78,   67,   86,   78,   64,   66,   76,   81,
          87,   93,   78,   86,   76,   87,   78,   99,  115,   69,
          85,   86,   70,  107,   78,   85,   65,   70,   99,   78,
          87,   61,   82,   98,   74,   75,   83,   75,   71,   79,
          80,   64,   75,   86,   68,   87,   82,   68,   79,   83], device='cuda:0')
score opp: tensor([  60,   58,   74,   79,

         65,  70,  70,  60], device='cuda:0')
-----------------------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------------------
score tm: tensor([  72,   63,   74,   72,   66,   74,   67,   54,   55,   76,
          76,   78,   58,   76,   70,   75,   78,   73,   80,   82,
          72,   80,   69,   83,   69,   91,   68,   84,   76,  110,
          82,   84,   85,   68,   73,   71,   82,   70,   71,   75,
          74,   68,   70,   74,   71,   97,   73,   89,   74,   97,
          70,   70,   67,   76,   73,   73,   79,   64,   77,   74,
          82,   68,   63,   77,   65,  102,   86,   83,   55,   81,
         112,   92,  105,   80,   99,   79,   79,   59,   85,   70,
          66,  103,   89,   74,   80,   85,   80,   77,   74,   81,
          80,   84,   99,   73,   92,  101,  101,   83,   77,   78], device='cuda:0')
score opp: tensor([  56,   61,   58,   60,

         70,  65,  70,  70], device='cuda:0')
-----------------------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------------------
score tm: tensor([  94,   72,   87,   66,   67,   86,   85,   85,   70,   81,
         105,   73,   79,   83,   71,   61,  102,   78,   75,  101,
          90,   92,   73,   79,   73,   79,   66,   80,   73,   84,
          61,   87,   76,   89,   76,   67,   62,   73,   75,   85,
          85,   70,   87,   74,   78,   82,  107,   84,   75,   88,
          72,   75,   83,   77,   80,   80,   76,   59,   88,   80,
          82,   89,   80,   78,   88,   94,   85,   93,   91,   82,
          85,   73,   83,   61,   71,   76,   76,   88,   87,   77,
          92,   90,   76,   65,   79,   77,   83,   82,   73,   79,
          72,   86,   87,   76,   81,   65,   87,   79,   71,   72], device='cuda:0')
score opp: tensor([  67,   63,   79,   60,

...

         70,  70,  70,  74], device='cuda:0')
-----------------------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------------------
score tm: tensor([  70,   72,   83,   87,   92,   84,   89,   70,   81,   73,
          64,   63,   59,   76,   60,   58,   65,   62,   55,   79,
          54,   80,   73,   64,   66,   83,   72,   86,   67,   62,
          63,   87,   94,   90,  103,   84,   74,   72,   69,   68,
          76,   69,   79,   68,  128,   65,   73,   74,   81,   83,
          75,   84,   83,   85,   85,   86,   69,   73,   77,   71,
          85,   81,   79,  101,   68,   75,   80,   80,   73,   72,
          93,   75,   69,   74,   76,   73,   74,   66,   69,   56,
          53,   79,   61,   76,   89,   82,   85,   72,   74,   67,
          72,   70,   92,   96,   60,   72,   52,   71,   70,   83], device='cuda:0')
score opp: tensor([ 64,  64,  68,  66,  66

         70,  70,  70,  70], device='cuda:0')
-----------------------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------------------
score tm: tensor([  84,  100,   71,   83,   80,   87,   90,   70,  100,   89,
         101,   61,   84,   59,   95,   89,   81,   80,   72,   88,
          89,   86,   85,   81,   69,   65,   67,   78,   75,   82,
          80,   77,   78,   70,   73,   80,  101,   84,   64,   84,
          70,   75,   72,   70,   71,   87,   71,   84,   70,   71,
          73,   81,   90,   60,   93,   73,   89,   78,   84,   72,
         112,  103,   93,   99,   69,   62,   67,   64,   81,   74,
          78,   62,   74,   67,   78,   50,   74,   77,   79,   84,
          68,   69,   87,   66,   78,   66,   77,   72,   79,   71,
          82,   79,   68,   65,   70,   62,   66,   88,   88,   74], device='cuda:0')
score opp: tensor([  72,   87,   63,   78,

         70,  70,  70,  70], device='cuda:0')
-----------------------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------------------
score tm: tensor([  80,   87,   71,   79,   72,   75,   60,   81,   77,   74,
          61,   65,   76,   88,   74,   58,   55,   77,   92,   69,
          89,   76,   78,   76,   76,   74,   83,   88,   84,   77,
          87,   72,   70,   59,   75,   64,   62,   76,   60,   64,
         106,   69,   76,   93,  104,   93,   86,   57,   74,   94,
          73,   87,  110,   74,   95,   81,   63,   82,   57,   89,
          80,   65,   96,   63,   90,   86,   82,   91,   90,   89,
          95,   68,   60,   67,   81,   72,   69,   87,   86,   68,
          76,   72,   87,   71,   82,   62,   70,   82,   66,   80,
          71,   85,   56,   81,   63,   78,   81,   84,   81,   84], device='cuda:0')
score opp: tensor([ 74,  58,  66,  70,  51

         65,  65,  65,  70], device='cuda:0')
-----------------------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------------------
score tm: tensor([  75,   79,   65,   84,   80,   69,   85,   65,   86,   76,
          66,   60,   80,   58,   76,   69,   93,  103,   81,   70,
          71,   63,   77,   73,   67,   75,   91,   69,   76,   93,
          78,   84,  109,   80,   83,   64,   71,   59,   87,   76,
          87,   53,   67,   88,   81,   77,   79,   65,   69,  102,
          89,   81,   90,   81,   79,   92,   89,   82,   70,   73,
          71,   67,   71,   70,   83,   83,   80,   60,   73,   65,
          97,   80,   86,   77,   95,   76,   90,   93,   76,   79,
          63,   78,   91,   86,   78,   86,   74,   96,   67,   56,
          65,   53,   66,   86,   84,   76,   62,   81,   73,   79], device='cuda:0')
score opp: tensor([  44,   65,   64,   65,

         70,  70,  70,  70], device='cuda:0')
-----------------------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------------------
score tm: tensor([  83,   76,   57,   81,   71,   68,   81,   86,   82,   72,
          74,  105,   85,   97,   98,   79,   80,   63,   93,   86,
          77,   72,   96,   74,   66,   60,   64,   84,   86,   78,
          62,   82,   85,   66,   94,   88,   86,   68,   88,   85,
          88,   66,   59,   91,   63,   57,   83,   80,   77,   79,
          60,   90,   74,   86,  101,   71,   72,  101,   98,   95,
          83,   85,   69,   63,   66,   67,   69,   68,   76,   82,
          57,   82,   57,   69,   73,   56,   74,   71,   77,   67,
          76,   67,   82,   87,   76,   67,   70,   70,   92,   74,
          77,   78,   78,   92,   86,   74,   94,   82,   87,   62], device='cuda:0')
score opp: tensor([ 67,  61,  52,  75,  55